<a href="https://colab.research.google.com/github/atzingan/DAEN690/blob/main/Forecasting%20Models/HarvestWeightForecastbySpeciesandFlyway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

#File stored in GitHub LFS
url = 'https://github.com/gjrsas/DAEN690/raw/main/Data/WingData.csv'

df = pd.read_csv(url)
print(df.shape)

<ipython-input-9-1071f08e51a4>:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


(5900720, 21)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

def load_data(url):
    return pd.read_csv(url)

def preprocess_data(data):
    data['Season'] = pd.to_datetime(data['Season'], format='%Y')
    data.sort_values('Season', inplace=True)
    return data

def fit_arima_model(train_series, order=(3, 1, 3)):
    arima_model = ARIMA(train_series.astype(np.float64), order=order)
    return arima_model.fit()

def calculate_metrics(actual, predicted):
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return rmse, mae, mape

def main():
    file_path = url
    data = load_data(url)

    # Group and sum the data
    grouped_data = data.groupby(['flyway_name', 'species_name', 'Season'])['harvest_weight'].sum().reset_index()

    # Get unique states and species
    flyways = grouped_data['flyway_name'].unique()
    species_list = grouped_data['species_name'].unique()

    # Create a DataFrame to store the results
    results_df = pd.DataFrame(columns=['Flyway Region', 'Species', 'Prediction', 'RMSE', 'MAE'])

    # Loop through each state and species combination
    for flyway in flyways:
        for species in species_list:
            specific_data = grouped_data[(grouped_data['flyway_name'] == flyway) & (grouped_data['species_name'] == species)]
            specific_data = preprocess_data(specific_data)

            # Split the data into training and testing sets
            if len(specific_data) > 5:  # Require at least 6 data points to split
                train_data = specific_data.iloc[:-2]  # Leave out the last 2 points for testing
                test_data = specific_data.iloc[-2:]

                time_series_train = train_data.set_index('Season')['harvest_weight']
                time_series_test = test_data.set_index('Season')['harvest_weight']

                try:
                    # Fit ARIMA model on training set
                    arima_results = fit_arima_model(time_series_train)

                    # Predict on the test set
                    predictions = arima_results.predict(start=time_series_test.index[0], end=time_series_test.index[-1])
                    rmse, mae, mape = calculate_metrics(time_series_test, predictions)

                    # Forecast the next year beyond the available data
                    forecast = arima_results.get_forecast(steps=1).predicted_mean.iloc[0]

                    # Append the results
                    results_df = results_df.append({
                        'Flyway Region': flyway,
                        'Species': species,
                        'Prediction': forecast,
                        'RMSE': rmse,
                        'MAE': mae,
                        'MAPE': mape
                    }, ignore_index=True)

                except Exception as e:
                    print(f"Could not fit ARIMA model for Flyway Region: {flyway}, Species: {species}. Error: {e}")

    # Save the results to an Excel file
    results_df.to_excel('arima_forecast_results.xlsx', index=False)

if __name__ == "__main__":
    main()

<ipython-input-11-509233d33dd2>:5: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(url)
<ipython-input-11-509233d33dd2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Season'] = pd.to_datetime(data['Season'], format='%Y')
<ipython-input-11-509233d33dd2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.sort_values('Season', inplace=True)
<ipython-input-11-509233d33dd2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

Could not fit ARIMA model for Flyway Region: Alaska, Species: Black Brant. Error: 'The `start` argument could not be matched to a location related to the index of the data.'


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
<ipython-input-11-509233d33dd2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Season'] = pd.to_datetime(data['Season'], format='%Y')
<ipython-input-11-509233d33dd2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.sort_values('Season', inplace=True)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: Valu

Could not fit ARIMA model for Flyway Region: Alaska, Species: Black Scoter. Error: 'The `start` argument could not be matched to a location related to the index of the data.'


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
<ipython-input-11-509233d33dd2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Season'] = pd.to_datetime(data['Season'], format='%Y')
<ipython-input-11-509233d33dd2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.sort_values('Season', inplace=True)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: Valu

...

Could not fit ARIMA model for Flyway Region: Pacific Flyway, Species: Fulvous Whistling Duck. Error: 'The `start` argument could not be matched to a location related to the index of the data.'


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
<ipython-input-11-509233d33dd2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.sort_values('Season', inplace=True)
<ipython-input-11-509233d33dd2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Season'] = pd.to_datetime(data['Season'], format='%Y')
<ipython-input-11-509233d33dd2>:9: SettingWithCopyWarning: 
A value is trying to be

Could not fit ARIMA model for Flyway Region: Pacific Flyway, Species: Mallard X Black Duck Hybrid. Error: 'The `start` argument could not be matched to a location related to the index of the data.'
Could not fit ARIMA model for Flyway Region: Pacific Flyway, Species: Muscovy. Error: LU decomposition error.


<ipython-input-11-509233d33dd2>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Season'] = pd.to_datetime(data['Season'], format='%Y')
<ipython-input-11-509233d33dd2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.sort_values('Season', inplace=True)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa

Could not fit ARIMA model for Flyway Region: Pacific Flyway, Species: Mexican-like Duck. Error: 'The `start` argument could not be matched to a location related to the index of the data.'
